In [1]:
using Pkg
Pkg.activate(".");
Pkg.status();

  Activating environment at `C:\Users\User\Documents\Graduate Files\Physics 215\Physics-215-Julia-Codes\Session 03\Project.toml`


      Status `C:\Users\User\Documents\Graduate Files\Physics 215\Physics-215-Julia-Codes\Session 03\Project.toml`
  [6e4b80f9] BenchmarkTools v1.2.0


# KR1: ```Number``` type hierarchy in Julia

For the start of the session, we look at the hierarchy of types in Julia under ```Number```. As was shown in the session slides, there are 514 variable data types in Julia, which is a lot! In the field of physics, the most important data type is ```Number```, as in physics, we deal with very many types of numbers. So it is instructive to look at the different data types of ```Number```s that are available to us in Julia.

To make it easier to see the ```Number``` type hierarchy in Julia, let us make use of the function shown to give us the ```subtype``` tree under ```Number```. The code used came from [this website](https://julia.quantecon.org/more_julia/generic_programming.html).

In [2]:
#  from https://github.com/JuliaLang/julia/issues/24741
function subtypetree(t, level=1, indent=4)
        if level == 1
            println(t)
        end
        for s in subtypes(t)
            println(join(fill(" ", level * indent)) * string(s))  # print type
            subtypetree(s, level+1, indent)  # recursively print the next type, indenting
        end
end;

With this code, when we now look at the ```subtypetree``` corresponding to the ```Number``` data type, we get.

In [3]:
subtypetree(Number)

Number
    Complex
    Real
        AbstractFloat
            BigFloat
            Float16
            Float32
            Float64
        AbstractIrrational
            Irrational
        Integer
            Bool
            Signed
                BigInt
                Int128
                Int16
                Int32
                Int64
                Int8
            Unsigned
                UInt128
                UInt16
                UInt32
                UInt64
                UInt8
        Rational


From the ```subtypetree``` of ```Number```, we see that there are two large subtypes -- ```Complex```, and ```Real```, and that ```Real``` data type has subtypes while ```Complex``` doesn't. This makes sense since the hallmark of a complex number is the addition of $\mathrm{i}=\sqrt{-1}$, and in [Julia](https://docs.julialang.org/en/v1/manual/complex-and-rational-numbers/#Complex-Numbers), this is done with the addition of ```im``` to the number as shown.

In [4]:
typeof(3 + 2im)

Complex{Int64}

Now we move to the ```Real``` subtype, since it has more subtypes. Out of the subtypes, of particular interest to us is the ```Rational``` and ```AbstractIrrational``` subtypes.

Apparently, Julia can handle [```Rational``` numbers](https://docs.julialang.org/en/v1/manual/complex-and-rational-numbers/#Rational-Numbers), mathematically defined as the ratio of two integers. We can define rational numbers through the use of ```//``` symbol, where the number preceding it is the numerator while the number succeeding it is the denominator.

In [5]:
typeof(22 // 7)

Rational{Int64}

This is a really nice feature as it makes Julia an easier language to do symbolic mathematics with. Python can do the same with the [```fractions``` module](https://docs.python.org/3/library/fractions.html), but to define a fraction, you have to make use of the ```Fraction()``` operator, which makes your code look a little messy, so it is nice that Julia makes it easier for us!

On the other hand, we have an ```Irrational``` number subtype, which apparently is the subtype where all the [mathematical constants](https://github.com/JuliaLang/julia/blob/master/base/mathconstants.jl) are stored in. It is nice that Julia has a specific data type to store its constants.

In [6]:
print(ℯ^1)
typeof(ℯ)

2.718281828459045

Irrational{:ℯ}

where ```ℯ``` is the Euler constant as shown.

At this point, we now go to the usual subtypes when dealing with ```Number```s -- ```AbstractFloat```, and ```Integer```.

As expected, there are different subtypes of ```AbstractFloat``` and ```Integer```, depending on the number of bits are using to store the number into the computer's memory -- 8 bits, 16 bits, 32 bits, 62 bits, and 128 bits for ```Integers```, and 16 bits, 32 bits, and 64 bits for ```AbstractFloat```. As seen in the subtype tree, we also see that there is a ```BigInt``` and ```BigFloat```. As we saw in Session 2, [```BigInt```](https://docs.julialang.org/en/v1/base/numbers/#Base.GMP.BigInt) is a datatype that handles arbitrarily large integers. And in the same vein, [```BigFloat```](https://docs.julialang.org/en/v1/base/numbers/#Base.MPFR.BigFloat) is a datatype that handles arbitarily large floating point numbers.

After going down the tree using ```subtype```, we note that we can go the other way around using ```supertype```. In the code shown, we look at the supertype chain starting from ```BigFloat```.

In [7]:
data_type = BigFloat

while data_type != Any
    println(data_type)
    data_type = supertype(data_type)
end

BigFloat
AbstractFloat
Real
Number


# KR2: Constructing ```struct```s

Reading about ```struct```s, the first impression I got was that it acted like a class -- it was an object that you could use to store data referring to a single object, like a simple point mass. Normally, when we define a point mass, we have to define its position, velocity, acceleration, and mass individually. With the help of a ```struct```, we can compile all of these properties into one ```struct``` named ```PointMass``` for example, so that when we want to define a point mass, we can just call store the values of its properties into one data type,  ```PointMass```.

With ```struct```s, like ```class```es, you can put functions inside them so that once you have the basic properties, you can get back a derived property that you are most likely going to use later on in your program. However, apparently this is [not necessarily true](https://www.fluentcpp.com/2017/06/13/the-real-difference-between-struct-class/) all the time. While you can do pretty much the same basic things with ```struct```s as you would with ```class```es, there are some limitations. In the raw sense, ```struct```s are static variables, which mean you cannot change their values once you initialize them.

To give an example, let us define our own ```struct``` influenced by music.

In [8]:
struct InstrumentTimbre
    amplitude_color::Vector{Float64}
    frequency_color::Vector{Int16}
end

Our struct here is used to store the timbre of a musical instrument. Every musical instrument has its own timbre, i.e. distinct characteristic sound, that can be measured by looking at the Fourier transform of the sound it makes. So to mimic it, we store the frequency/harmonics it produces, and their corresponding amplitudes. Further, we define amplitudes to be a floating point number, while frequencies to be only integers from 1 Hz to 32767 Hz, which is more than enough for us.

Another thing I noticed while making this is that once you defined the struct, you can no longer modify it, so let's say I defined my struct to take in ```Vectors```, but decided that one should have a vector of ```Float64``` numbers while the other, ```Int16```. To do so, I have to restart the kernel and run it because after running it once and naively modifying it, Julia will throw an exception saying I have an invalid redefinition of my struct. One extra thing to take note of!

Ok, let's try to use our newly made ```struct```. Let us define an imaginary instrument with amplitude color of ```amp``` and frequency color of ```freq```, and define a variable ```theo_inst``` that has these properties.

In [9]:
amp = rand(32767)
freq = 1:32767

theo_inst = InstrumentTimbre(amp, freq)

InstrumentTimbre([0.6238263848318208, 0.7732939846459512, 0.4816756201854966, 0.5222909210989564, 0.6963331960744938, 0.7245521028112334, 0.03991443000558159, 0.46907895891340434, 0.32172200295571973, 0.022215288543705247  …  0.0736532671743182, 0.021830195885000014, 0.701099661321174, 0.018709718304898537, 0.1677910500565789, 0.05214052226234922, 0.30315646409176655, 0.020500369976524713, 0.9068433886857288, 0.800488465736581], Int16[1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  32758, 32759, 32760, 32761, 32762, 32763, 32764, 32765, 32766, 32767])

So that when we want to retrieve the properties of this instrument such as the ```amplitude color```, we can simply call.

In [10]:
theo_inst.amplitude_color

32767-element Vector{Float64}:
 0.6238263848318208
 0.7732939846459512
 0.4816756201854966
 0.5222909210989564
 0.6963331960744938
 0.7245521028112334
 0.03991443000558159
 0.46907895891340434
 0.32172200295571973
 0.022215288543705247
 0.830544163138746
 0.49790659887352495
 0.6217617741831618
 ⋮
 0.6385428746760016
 0.8117271555085341
 0.0736532671743182
 0.021830195885000014
 0.701099661321174
 0.018709718304898537
 0.1677910500565789
 0.05214052226234922
 0.30315646409176655
 0.020500369976524713
 0.9068433886857288
 0.800488465736581

Like classes, we can define multiple theoretical instruments using our ```struct```, however we cannot modify the properties of these theoretical instruments once they have been instantiated. For example, let us try to change the ```amplitude_color``` of ```theo_inst```.

In [11]:
new_amp = rand(32767)

theo_inst.amplitude_color = new_amp

LoadError: setfield! immutable struct of type InstrumentTimbre cannot be changed

Which is because the object ```theo_inst``` is stored into a specific location in memory and as was shown in the session, Julia does not allow changes in values stored in a specific reference location.

To alleviate this, we can define a ```mutable struct``` version of the ```struct``` as follows.

In [12]:
mutable struct InstrumentTimbreMut
    amplitude_color::Vector{Float64}
    frequency_color::Vector{Int16}
end

Then defining a new theoretical instrument ```theo_inst_2```,

In [13]:
amp_mut = rand(32767)
freq_mut = 1:32767

theo_inst_2 = InstrumentTimbreMut(amp_mut, freq_mut)

InstrumentTimbreMut([0.0489925625257821, 0.17148275951563452, 0.6960605632155494, 0.7534333237795474, 0.49384647095667633, 0.7223317041041089, 0.18056696603987477, 0.020469676496909894, 0.1679117857717558, 0.25042652320391623  …  0.7138683903672491, 0.864768556914878, 0.21565874653234962, 0.19285136636757083, 0.7123385967299003, 0.9730694521683287, 0.7279875361543879, 0.46340570323229535, 0.3668342225860377, 0.053074118079940336], Int16[1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  32758, 32759, 32760, 32761, 32762, 32763, 32764, 32765, 32766, 32767])

Then change the values of ```amp_mut```,

In [14]:
amp_mut_2 = 2.0*rand(32767)

theo_inst_2.amplitude_color = amp_mut_2

theo_inst_2

InstrumentTimbreMut([0.9353602941387105, 0.18886445795885143, 0.0238022985802826, 1.741434608159019, 1.41412866665171, 1.3949904726382396, 0.8338254376575973, 0.38126462803970984, 0.29385285142289685, 0.5511201150370972  …  1.3924511573544542, 0.8877633228431683, 0.8913355847271829, 0.8491263857791211, 0.6904269983207554, 0.11204632537138126, 0.40324674991809406, 1.3490139087551785, 1.5425813629360174, 1.2874676198511392], Int16[1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  32758, 32759, 32760, 32761, 32762, 32763, 32764, 32765, 32766, 32767])

Then Julia allows us to change these properties easily. However, this appear to defeat the purpose of ```struct``` since if you would want to do this, then it would be better to define a class instead. From this example, we see that ```struct```s can be used to create lookup tables to make it easier lookup values that you would want to get back to anytime in your code.

There is another type of ```struct``` that you can make in Julia, that is the parametrized ```struct```. With this ```struct```, you may define an object that can take in any type of data (or can be restricted to one supertype), and Julia would be able to determine which object property will store which.

When we try to parametrize our ```InstrumentTimbre``` ```struct```, we see that it won't best to do so since it requires a specific data type. In this case, let us define a different ```struct``` that allows this. Thus, to demonstrate this, let us define another ```struct``` that stores the momentum and energy of an object.

In [15]:
struct MomentumEnergyPar{T<:Number}
    momentum::Vector{T}
    energy::T
end

Doing so makes the most out of parametrized ```struct```s as it defines a ```struct``` with object properties with abstract data types, thus when we instantiate it, we immediately see which variable data type will go to which object property.

In [16]:
particle_momentum = 5.0*rand(2)
particle_energy = 3.25

particle_momenergy = MomentumEnergyPar(particle_momentum, particle_energy)

MomentumEnergyPar{Float64}([1.2387781986318869, 4.2142213169565], 3.25)

# KR3: Type Inferencing in Julia

Like in Python, Julia can immediately infer the data type you are using depending on the expression or values you input in your expressions. An immediate example would be as follows.

In [17]:
[x for x in 1:5]

5-element Vector{Int64}:
 1
 2
 3
 4
 5

In [18]:
[1/x for x in 1:5]

5-element Vector{Float64}:
 1.0
 0.5
 0.3333333333333333
 0.25
 0.2

So notice that while we were inputting integers in our generator expressions, Julia was immediately able to infer the data type we are expected to get, that is for ```x``` we will expect an integer, while for ```1/x``` we will expect a floating point number.

# KR4 and KR5: Inherent Type-Stability in Julia

Now let us see how we can have type-stability for functions in Julia. As stated in [Type Stability in Julia](https://arxiv.org/pdf/2109.01950.pdf) type-stability is the case when the type of the output of a function cannot vary depending on the values of the inputs. In layman's terms, this means that the a function is type-stable if the data type of the output of a function is always the same, independent of the data type of the values that are inputted to it.

Let us make an example of a function with type-instability. To demonstrate, let us define a shifted absolute function ```shift_abs(x, a)``` so that
\begin{equation}
    \mathrm{shift\_abs}(x, a) =
    \begin{cases}
        0 & |x|<a \\
        |x|-a & |x|\geq a
    \end{cases}
\end{equation}

In [19]:
shift_abs(x, a) = abs(x) < a ? 0 : abs(x) - a;

We want the function to always output a floating point number. However, when we try certain numbers, we get the following result.

In [20]:
println("Given x=4 and a=5, shift_abs(x, a) = $(shift_abs(4, 5)), with a data type of $(typeof(shift_abs(4, 5))).")
println("Given x=5 and a=5, shift_abs(x, a) = $(shift_abs(5, 5)), with a data type of $(typeof(shift_abs(5, 5))).")
println("Given x=6 and a=5, shift_abs(x, a) = $(shift_abs(6, 5)), with a data type of $(typeof(shift_abs(6, 5))).")

Given x=4 and a=5, shift_abs(x, a) = 0, with a data type of Int64.
Given x=5 and a=5, shift_abs(x, a) = 0, with a data type of Int64.
Given x=6 and a=5, shift_abs(x, a) = 1, with a data type of Int64.


Which is not what we want! Furthermore, if we go with the other way around, we would get a different result.

In [21]:
println("Given x=4 and a=5.0, shift_abs(x, a) = $(shift_abs(4, 5.0)), with a data type of $(typeof(shift_abs(4, 5.0))).")
println("Given x=5 and a=5.0, shift_abs(x, a) = $(shift_abs(5, 5.0)), with a data type of $(typeof(shift_abs(5, 5.0))).")
println("Given x=6 and a=5.0, shift_abs(x, a) = $(shift_abs(6, 5.0)), with a data type of $(typeof(shift_abs(6, 5.0))).")

Given x=4 and a=5.0, shift_abs(x, a) = 0, with a data type of Int64.
Given x=5 and a=5.0, shift_abs(x, a) = 0.0, with a data type of Float64.
Given x=6 and a=5.0, shift_abs(x, a) = 1.0, with a data type of Float64.


So clearly our function is type-unstable! To see it clearly, we use the ```@code_warntype``` macro.

In [22]:
@code_warntype shift_abs(5, 5)

Variables
  #self#::Core.Const(shift_abs)
  x::Int64
  a::Int64

Body::Int64
1 ─ %1 = Main.abs(x)::Int64
│   %2 = (%1 < a)::Bool
└──      goto #3 if not %2
2 ─      return 0
3 ─ %5 = Main.abs(x)::Int64
│   %6 = (%5 - a)::Int64
└──      return %6


In [23]:
@code_warntype shift_abs(4, 5.0)

Variables
  #self#::Core.Const(shift_abs)
  x::Int64
  a::Float64

Body::Union{Float64, Int64}
1 ─ %1 = Main.abs(x)::Int64
│   %2 = (%1 < a)::Bool
└──      goto #3 if not %2
2 ─      return 0
3 ─ %5 = Main.abs(x)::Int64
│   %6 = (%5 - a)::Float64
└──      return %6


In [24]:
@code_warntype shift_abs(5, 5.0)

Variables
  #self#::Core.Const(shift_abs)
  x::Int64
  a::Float64

Body::Union{Float64, Int64}
1 ─ %1 = Main.abs(x)::Int64
│   %2 = (%1 < a)::Bool
└──      goto #3 if not %2
2 ─      return 0
3 ─ %5 = Main.abs(x)::Int64
│   %6 = (%5 - a)::Float64
└──      return %6


So it is further clear where the type-instability is coming from. It is from the conversion of data type when the function is being evaluated. To remedy this, then it would be better to define the variables as floating point numbers in the function so that the function will always work with floating point numbers.

In [25]:
shift_abs_stable(x, a) = abs(x) < a ? 0.0 : abs(x) - 1.0*a;

So that now, when we evaluate the function for the same parameters, we get:

In [26]:
println("Given x=4 and a=5, shift_abs_stable(x, a) = $(shift_abs_stable(4, 5)), with a data type of $(typeof(shift_abs_stable(4, 5))).")
println("Given x=5 and a=5, shift_abs_stable(x, a) = $(shift_abs_stable(5, 5)), with a data type of $(typeof(shift_abs_stable(5, 5))).")
println("Given x=6 and a=5, shift_abs_stable(x, a) = $(shift_abs_stable(6, 5)), with a data type of $(typeof(shift_abs_stable(6, 5))).")

println("Given x=4 and a=5.0, shift_abs_stable(x, a) = $(shift_abs_stable(4, 5.0)), with a data type of $(typeof(shift_abs_stable(4, 5.0))).")
println("Given x=5 and a=5.0, shift_abs_stable(x, a) = $(shift_abs_stable(5, 5.0)), with a data type of $(typeof(shift_abs_stable(5, 5.0))).")
println("Given x=6 and a=5.0, shift_abs_stable(x, a) = $(shift_abs_stable(6, 5.0)), with a data type of $(typeof(shift_abs_stable(6, 5.0))).")

Given x=4 and a=5, shift_abs_stable(x, a) = 0.0, with a data type of Float64.
Given x=5 and a=5, shift_abs_stable(x, a) = 0.0, with a data type of Float64.
Given x=6 and a=5, shift_abs_stable(x, a) = 1.0, with a data type of Float64.
Given x=4 and a=5.0, shift_abs_stable(x, a) = 0.0, with a data type of Float64.
Given x=5 and a=5.0, shift_abs_stable(x, a) = 0.0, with a data type of Float64.
Given x=6 and a=5.0, shift_abs_stable(x, a) = 1.0, with a data type of Float64.


Then running the ```@code_warntype``` macro, we have:

In [27]:
@code_warntype shift_abs_stable(5, 5)

Variables
  #self#::Core.Const(shift_abs_stable)
  x::Int64
  a::Int64

Body::Float64
1 ─ %1 = Main.abs(x)::Int64
│   %2 = (%1 < a)::Bool
└──      goto #3 if not %2
2 ─      return 0.0
3 ─ %5 = Main.abs(x)::Int64
│   %6 = (1.0 * a)::Float64
│   %7 = (%5 - %6)::Float64
└──      return %7


In [28]:
@code_warntype shift_abs_stable(4, 5.0)

Variables
  #self#::Core.Const(shift_abs_stable)
  x::Int64
  a::Float64

Body::Float64
1 ─ %1 = Main.abs(x)::Int64
│   %2 = (%1 < a)::Bool
└──      goto #3 if not %2
2 ─      return 0.0
3 ─ %5 = Main.abs(x)::Int64
│   %6 = (1.0 * a)::Float64
│   %7 = (%5 - %6)::Float64
└──      return %7


In [29]:
@code_warntype shift_abs_stable(5, 5.0)

Variables
  #self#::Core.Const(shift_abs_stable)
  x::Int64
  a::Float64

Body::Float64
1 ─ %1 = Main.abs(x)::Int64
│   %2 = (%1 < a)::Bool
└──      goto #3 if not %2
2 ─      return 0.0
3 ─ %5 = Main.abs(x)::Int64
│   %6 = (1.0 * a)::Float64
│   %7 = (%5 - %6)::Float64
└──      return %7


Which is what we want.

# KR6 Arrays with ```AbstractType``` values

Lastly, let us investigate how our code would perform if we had used ```AbstractType``` values vs specific type values. First, we define two arrays:

In [30]:
int_array = Int64[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
num_array = Number[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20];

Let us pass these functions to the ```sum()``` function in Julia. Doing so, we get:

In [31]:
println("The sum of integers is $(sum(int_array)) with data type $(typeof(sum(int_array))).")
println("The sum of numbers is $(sum(num_array)) with data type $(typeof(sum(num_array))).")

The sum of integers is 210 with data type Int64.
The sum of numbers is 210 with data type Int64.


So we get the same result good! But let us see how it will work when we benchmark it.

In [32]:
using BenchmarkTools

perf_int = @benchmark sum($int_array)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  5.100 ns … 9.900 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     5.200 ns             ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.182 ns ± 0.115 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▇                 █                  ▁                  ▂ ▁
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ █
  5.1 ns      Histogram: log(frequency) by time      5.4 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [33]:
perf_num = @benchmark sum($num_array)

BenchmarkTools.Trial: 10000 samples with 600 evaluations.
 Range (min … max):  200.833 ns … 322.333 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     211.833 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   212.030 ns ±   3.285 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                      ▁  ▂ ▅▇▃▇█▄▅▅▂                         ▁▁ ▂
  ▃▁▁▁▁▁▁▃▆▄▁▁▃▁▁▁▆▇▅▆█▅▇█▆████████████▆▆▆▆▆▆▆▅▆▄▄▄▄▆▄▄▅▆▅▆█▆██ █
  201 ns        Histogram: log(frequency) by time        224 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

So that taking the ratio of the median time of evaluation between the two arrays, we have:

In [34]:
println("Ratio of time to evaluate with num_array with that for int_array is $(median(perf_num.times)/median(perf_int.times)).")

Ratio of time to evaluate with num_array with that for int_array is 40.73717948717949.


Thus it takes 40 times as much time for an abstract type array to be evaluated compared to a specific type array!